In [18]:
from torchvision import transforms
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from yullab.models import MLP, CNN, RNN, LSTM
from yullab.datasets import FashionDataset
from yullab.trainer import train, test
from time import time
import getpass

ModuleNotFoundError: No module named 'torchvision'

In [ ]:
username = getpass.getuser()
fashion_train_filename = f'/Users/{username}/data/fashion_mnist/fashion-mnist_train.csv'
fashion_val_filename = f'/Users/{username}/data/fashion_mnist/fashion-mnist_test.csv'

text_labels = ['t-shirt', 'trouser', 'pullover', 'dress', 'coat',
                   'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot']

In [ ]:
# for cnn, mlp
start_time = time()
train_transforms = transforms.Compose([transforms.ToTensor(),
                                       transforms.Normalize(0, 1)])

train_dataset = FashionDataset(fashion_train_filename, train_transforms)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataset = FashionDataset(fashion_val_filename, train_transforms)
val_loader = DataLoader(val_dataset,
                        batch_size=len(val_dataset),
                        shuffle=False)
# net = MLP()
#net = CNN()
net = LSTM()

lr = 0.005
n_epochs = 3

optimizer = optim.Adam(net.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

train_losses = train(train_loader, net, criterion, optimizer, n_epochs)
test_loss, accuracy  = test(val_loader, net, criterion)
end_time = time()
print(end_time-start_time)

Epoch: 1/3..  Training Loss: 0.617.. 
Epoch: 2/3..  Training Loss: 0.381.. 
Epoch: 3/3..  Training Loss: 0.334.. 
Test Loss: 0.324..  Test Accuracy: 0.881
217.67272424697876


In [ ]:
%%time
from yullab.models import VAE
import torch.nn.functional as F

train_transforms = transforms.Compose([transforms.ToTensor(),
                                       transforms.Normalize(0, 1)])

train_dataset = FashionDataset(fashion_train_filename, train_transforms)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataset = FashionDataset(fashion_val_filename, train_transforms)
val_loader = DataLoader(val_dataset,
                        batch_size=len(val_dataset),
                        shuffle=False)

model = VAE(784, 200, 20)

optimizer = optim.Adam(net.parameters(), lr=lr)
criterion = nn.BCELoss()

train_losses = []
for epoch in range(n_epochs):
    running_loss = 0
    for images, _ in train_loader:
        optimizer.zero_grad()
        images = images.reshape(32, 1, -1)

        outputs, mu, sigma = model(images)

        bce = criterion(images, outputs)
        kl_div = 0.5 * torch.sum(mu.pow(2) + sigma)
        loss = bce + kl_div

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    train_losses.append(running_loss/len(train_loader))
    print("Epoch: {}/{}.. ".format(epoch + 1, n_epochs),
          "Training Loss: {:.3f}.. ".format(running_loss / len(train_loader)))

Epoch: 1/3..  Training Loss: 19.994.. 
Epoch: 2/3..  Training Loss: 19.993.. 
Epoch: 3/3..  Training Loss: 19.993.. 
CPU times: user 2min 35s, sys: 3min 3s, total: 5min 38s
Wall time: 55.4 s


In [ ]:
images.reshape(32, 1, -1)

tensor([[[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]]])

In [ ]:
import pandas as pd

csv_filename = '/Users/rimiiii/csv파일위치/csv_filename'

# 파일 불러오기 
df = pd.read_csv(csv_filename)

In [ ]:
df = pd.read_csv('coffee_menu.csv')
df.groupby('shop_name').mean()

/var/folders/w5/d9f3nc_j45x92pn9th1bphzw0000gn/T/ipykernel_33298/3654110756.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby('shop_name').mean()


,price
shop_name,
A,4375.0
B,4500.0


In [ ]:
df.to_csv('test.csv')

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame({'shop_name': ['A', 'A', 'A', 'A', 'B', 'B', 'B'], 'menu': ['아이스아메리카노', '밀크티', '딸기라떼', '녹차라뗴', '아이스아메리카노', '밀크티', '카페라떼'], 'price': [3000, 4000, 5000, 5500, 4000, 5000, 4500]})
# df.to_csv('저장할파일이름.csv')
df.groupby('shop_name').mean()

/var/folders/w5/d9f3nc_j45x92pn9th1bphzw0000gn/T/ipykernel_48905/249245230.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby('shop_name').mean()


,price
shop_name,
A,4375.0
B,4500.0


In [ ]:
print(df.iloc[3])
print(df.iloc[3,2])

shop_name       A
menu         녹차라뗴
price        5500
Name: 3, dtype: object
5500


In [ ]:
df = pd.DataFrame({'menu': ['아이스아메리카노', '밀크티', '딸기라떼'], 
									 'price': [3000, 4000, 5000]})
df.price >= 4000
#0    False
#1     True
#2     True
#Name: price, dtype: bool

df[df.price >= 4000]

,menu,price
1,밀크티,4000
2,딸기라떼,5000


In [ ]:
df = pd.DataFrame({'shop_name': ['A', 'A', 'A', 'A', 'B', 'B', 'B'], 'menu': ['아이스아메리카노', '밀크티', '딸기라떼', '녹차라뗴', '아이스아메리카노', '밀크티', '카페라떼'], 'price': [3000, 4000, 5000, 5500, 4000, 5000, 4500]})
df.groupby('shop_name').mean()

def add_ice_price(menu, price):
	return price+500 if '아이스' in menu else price

df['new_price'] = df.apply(lambda x: add_ice_price(x.menu, x.price), axis=1)
df

/var/folders/w5/d9f3nc_j45x92pn9th1bphzw0000gn/T/ipykernel_48905/941435925.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby('shop_name').mean()


,shop_name,menu,price,new_price
0,A,아이스아메리카노,3000,3500
1,A,밀크티,4000,4000
2,A,딸기라떼,5000,5000
3,A,녹차라뗴,5500,5500
4,B,아이스아메리카노,4000,4500
5,B,밀크티,5000,5000
6,B,카페라떼,4500,4500


In [17]:
# df.rename(columns={'menu': '메뉴', 'price': '가격'})
df.rename(str.upper, axis='columns')
# df

,SHOP_NAME,MENU,PRICE,NEW_PRICE
0,A,아이스아메리카노,3000,3500
1,A,밀크티,4000,4000
2,A,딸기라떼,5000,5000
3,A,녹차라뗴,5500,5500
4,B,아이스아메리카노,4000,4500
5,B,밀크티,5000,5000
6,B,카페라떼,4500,4500
